In [1]:
import pandas as pd
from pandas import DataFrame, Series
import requests
import re
import lxml
from bs4 import BeautifulSoup
import numpy as np

### 크롤링을 하기 위해 필요한 기본 사항
실제로 데이터를 가져올 웹사이트 주소를 chart_url과 같이 변수에 할당해준다.
서버에 Request를 보낼 때 안정적인 데이터 확보를 위해 User Agent를 헤더에 담아 URL주소와 headers를 같이 요청한다.
headers는 선택사항이지만 크롤링을 하기 위해 꼭 넣어서 서버에 요청하는 것을 권장
그리고 headers(User Agent)는 브라우저 중에서 호환성이 좋은 FireFox, Chrome을 추천

In [2]:
chart_url = 'https://music.bugs.co.kr/chart'
headers = {
    'User_agent': 'Mozilla/5.0 (Macintosh; Intel Mac os X x.y; rv:10.0) Gecko/20100101 Firefox/10.0'
}

### requests 모듈을 사용하여 Response 200 코드 받기
- requests 모듈을 사용하여 chart_url을 가져오게 되면 HTTP Response의 응답을 받게 된다. Response 200을 받으면 데이터 파싱이 가능한 상태
```
> res = requests.get(chart_url, headers)<br>
> res
# output <Response [200]>
```

In [3]:
res = requests.get(chart_url, headers=headers)
html = res.text

### pandas를 사용하여 크롤링
#### pandas의 자료구조인 data_frame과 series로 크롤링
> DataFrame은 2차적 데이터 구조 2개 이상의 column이 존재하면 DataFrame이라고 생각

>Series는 1차적 데이터 구조로 1개의 column을 보여주며, Series를 사용하는 이유는 2개의 데이터베이스를 합칠 경우에 index의 순서가 달라도 스마트하게 동일한 index끼리의 처리를 할 수 있기 때문

In [4]:
type(pd.read_html(html)) # 데이터베이스 테이블이 list타입임을 확인

list

In [5]:
len(pd.read_html(html)) # 데이터베이스 테이블(리스트)가 3개임을 확인

3

In [6]:
chart_df = pd.read_html(html)[0] # 3개의 리스트 중에서 첫번째 리스트 값을 크롤링
chart_df

,Unnamed: 0,순위,Unnamed: 2,Unnamed: 3,곡,아티스트,앨범,듣기,재생목록,내앨범,다운,영상,기타
0,NaN,1 0변동없음,NaN,곡정보,멍청이(twit),화사(Hwa Sa),멍청이(twit),듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
1,NaN,2 0변동없음,NaN,곡정보,달라달라,ITZY (있지),IT'z Different,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
2,NaN,3 0변동없음,NaN,곡정보,오롯이,헤이즈(HEIZE),오롯이,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능
3,NaN,4 0변동없음,NaN,곡정보,이 노래가 클럽에서 나온다면,우디(Woody),이 노래가 클럽에서 나온다면,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
4,NaN,5 0변동없음,NaN,곡정보,이별을 걷다 (A Walk To Goodbye),황치열,The Four Seasons,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
5,NaN,6 1 계단 상승,NaN,곡정보,벌써 12시,청하,벌써 12시,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
6,NaN,7 1 계단 하락,NaN,곡정보,7 rings,Ariana Grande(아리아나 그란데),7 rings,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
7,NaN,8 1 계단 상승,NaN,곡정보,신청곡 (Feat. SUGA of BTS),이소라,신청곡 (Feat. SUGA of BTS),듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
8,NaN,9 1 계단 하락,NaN,곡정보,오랜만이야 (Feat. Zion.T),로꼬,HELLO,듣기,재생목록에 추가,내 앨범에 담기,다운로드,영상 재생,기타 기능
9,NaN,10 0변동없음,NaN,곡정보,띵 (Prod. By 기리보이),Jvcki Wai Jvcki Wai,Dingo X Indigo Music,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능


## 크롤링한 데이터 프레임의 행과 열을 파악하는 방법 (.shape)

In [7]:
chart_df.columns

Index(['Unnamed: 0', '순위', 'Unnamed: 2', 'Unnamed: 3', '곡', '아티스트', '앨범', '듣기',
       '재생목록', '내앨범', '다운', '영상', '기타'],
      dtype='object')

In [8]:
pd.DataFrame(chart_df, columns=['순위', '곡', '아티스트', '앨범']).set_index('곡')

,순위,아티스트,앨범
곡,,,
멍청이(twit),1 0변동없음,화사(Hwa Sa),멍청이(twit)
달라달라,2 0변동없음,ITZY (있지),IT'z Different
오롯이,3 0변동없음,헤이즈(HEIZE),오롯이
이 노래가 클럽에서 나온다면,4 0변동없음,우디(Woody),이 노래가 클럽에서 나온다면
이별을 걷다 (A Walk To Goodbye),5 0변동없음,황치열,The Four Seasons
벌써 12시,6 1 계단 상승,청하,벌써 12시
7 rings,7 1 계단 하락,Ariana Grande(아리아나 그란데),7 rings
신청곡 (Feat. SUGA of BTS),8 1 계단 상승,이소라,신청곡 (Feat. SUGA of BTS)
오랜만이야 (Feat. Zion.T),9 1 계단 하락,로꼬,HELLO


### 데이터 프레임에 '좋아요'필드를 추가하고 head, tail을 사용하기
- .head()는 기본적으로 상위 5개를 출력 따로 옵션을 주고 싶다면() 안에 옵션을 주면된다.
- .tail()또한 기본적으로 하위 5개를 출력 옵션은 위와 같다

In [9]:
color_dict = {'blue': '파란색', 'red': '빨간색', 'yellow': '노란색'}

In [10]:
color_dict

{'blue': '파란색', 'red': '빨간색', 'yellow': '노란색'}

In [11]:
color_dict['black'] = '검정색'

In [12]:
color_dict

{'blue': '파란색', 'red': '빨간색', 'yellow': '노란색', 'black': '검정색'}

In [13]:
chart_df['좋아요'] = 100

In [14]:
# 정렬 기준에 따라서 레코드값을 1~101까지 설정
# 만약 데이터프레임과 설정하려는 레코드값이 다를 경우 length에러가 발생
# chart_df['좋아요'] = range(1, 30) <-- length에러

chart_df['좋아요'] = range(1, 101)

In [15]:
chart_df.head(10)

,Unnamed: 0,순위,Unnamed: 2,Unnamed: 3,곡,아티스트,앨범,듣기,재생목록,내앨범,다운,영상,기타,좋아요
0,NaN,1 0변동없음,NaN,곡정보,멍청이(twit),화사(Hwa Sa),멍청이(twit),듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,1
1,NaN,2 0변동없음,NaN,곡정보,달라달라,ITZY (있지),IT'z Different,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,2
2,NaN,3 0변동없음,NaN,곡정보,오롯이,헤이즈(HEIZE),오롯이,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,3
3,NaN,4 0변동없음,NaN,곡정보,이 노래가 클럽에서 나온다면,우디(Woody),이 노래가 클럽에서 나온다면,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,4
4,NaN,5 0변동없음,NaN,곡정보,이별을 걷다 (A Walk To Goodbye),황치열,The Four Seasons,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,5
5,NaN,6 1 계단 상승,NaN,곡정보,벌써 12시,청하,벌써 12시,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,6
6,NaN,7 1 계단 하락,NaN,곡정보,7 rings,Ariana Grande(아리아나 그란데),7 rings,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,7
7,NaN,8 1 계단 상승,NaN,곡정보,신청곡 (Feat. SUGA of BTS),이소라,신청곡 (Feat. SUGA of BTS),듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,8
8,NaN,9 1 계단 하락,NaN,곡정보,오랜만이야 (Feat. Zion.T),로꼬,HELLO,듣기,재생목록에 추가,내 앨범에 담기,다운로드,영상 재생,기타 기능,9
9,NaN,10 0변동없음,NaN,곡정보,띵 (Prod. By 기리보이),Jvcki Wai Jvcki Wai,Dingo X Indigo Music,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,10


### .head()옵션
head 옵션을 사용하면 상위의 지정한 갯수만큼의 필드값을 추출할 수 있다.
- chart_df.head() # 기본은 5개
- chart_df.head(50) # 50개의 필드 추출

In [16]:
chart_df.head()

,Unnamed: 0,순위,Unnamed: 2,Unnamed: 3,곡,아티스트,앨범,듣기,재생목록,내앨범,다운,영상,기타,좋아요
0,NaN,1 0변동없음,NaN,곡정보,멍청이(twit),화사(Hwa Sa),멍청이(twit),듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,1
1,NaN,2 0변동없음,NaN,곡정보,달라달라,ITZY (있지),IT'z Different,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,2
2,NaN,3 0변동없음,NaN,곡정보,오롯이,헤이즈(HEIZE),오롯이,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,3
3,NaN,4 0변동없음,NaN,곡정보,이 노래가 클럽에서 나온다면,우디(Woody),이 노래가 클럽에서 나온다면,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,4
4,NaN,5 0변동없음,NaN,곡정보,이별을 걷다 (A Walk To Goodbye),황치열,The Four Seasons,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,5


### .tail() 옵션
#### tail 옵션을 사용하면 하위의 지정한 갯수만큼의 필드값을 추출할 수 있다.
- chart_df.tail() # 기본은 5개
- chart_df.tail(50) # 50개의 필드 추출

In [17]:
chart_df.tail()

,Unnamed: 0,순위,Unnamed: 2,Unnamed: 3,곡,아티스트,앨범,듣기,재생목록,내앨범,다운,영상,기타,좋아요
95,NaN,96 2 계단 하락,NaN,곡정보,꿈처럼 내린,다비치,뷰티 인사이드 (JTBC 월화드라마) OST - Part.3,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,96
96,NaN,97 2 계단 상승,NaN,곡정보,가을 안부,먼데이 키즈(Monday Kiz),가을 안부,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,97
97,NaN,98 22 계단 하락,NaN,곡정보,Make it count,첸 (CHEN),진심이 닿다 (tvN 수목드라마) OST - Part.1,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,98
98,NaN,99 6 계단 상승,NaN,곡정보,PIRI,드림캐쳐(Dreamcatcher),The End of Nightmare,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,99
99,NaN,100 31 계단 상승,NaN,곡정보,[19금] IndiGO,저스디스(JUSTHIS) 저스디스(JUSTHIS),IM,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,100


### 레코드값을 이용하여 오름차순, 내림차순하기
기본옵션은 True로 ascending옵션을 명시할 필요는 없다.

In [18]:
a = chart_df.sort_values('아티스트', ascending=False)
a[['곡', '아티스트']]

,곡,아티스트
4,이별을 걷다 (A Walk To Goodbye),황치열
54,그대가 내 안에 박혔다,황치열
0,멍청이(twit),화사(Hwa Sa)
2,오롯이,헤이즈(HEIZE)
29,첫눈에,헤이즈(HEIZE)
76,흔한 이별,허각
18,We all lie,하진
13,신용재,하은(라코스테남)
20,너를 만나,폴킴(Paul Kim)
17,"모든 날, 모든 순간 (Every day, Every Moment)",폴킴(Paul Kim)


### 좋아요 기준 50개를 넘는 레코드값만 출력하기

In [19]:
mask = chart_df['좋아요'] > 50
chart_df[mask]

,Unnamed: 0,순위,Unnamed: 2,Unnamed: 3,곡,아티스트,앨범,듣기,재생목록,내앨범,다운,영상,기타,좋아요
50,NaN,51 22 계단 상승,NaN,곡정보,Home,세븐틴(Seventeen),SEVENTEEN 6TH MINI ALBUM 'YOU MADE MY DAWN',듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,51
51,NaN,52 4 계단 하락,NaN,곡정보,IDOL,방탄소년단,LOVE YOURSELF 結 ‘Answer’,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,52
52,NaN,53 3 계단 상승,NaN,곡정보,Dance The Night Away,TWICE (트와이스),Summer Nights,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,53
53,NaN,54 6 계단 상승,NaN,곡정보,Wind flower,마마무(Mamamoo),BLUE;S,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,54
54,NaN,55 10 계단 하락,NaN,곡정보,그대가 내 안에 박혔다,황치열,그대가 내 안에 박혔다,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,55
55,NaN,56 5 계단 하락,NaN,곡정보,밤편지,아이유(IU),밤편지,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,56
56,NaN,57 0변동없음,NaN,곡정보,그 남자를 떠나,지오디(god),THEN & NOW,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,57
57,NaN,58 3 계단 하락,NaN,곡정보,Sunflower (Spider-Man: Into the Spider-Verse),Post Malone(포스트 말론) Post Malone(포스트 말론),스파이더맨: 뉴 유니버스 (Spider-Man: Into the Spider-Ver...,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,58
58,NaN,59 13 계단 상승,NaN,곡정보,지나오다,닐로(Nil_O),About You,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,59
59,NaN,60 20 계단 상승,NaN,곡정보,하루도 그대를 사랑하지 않은 적이 없었다,임창정,하루도 그대를 사랑하지 않은 적이 없었다,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,60


### 1차적 데이터 구조를 가진 Series
Series를 사용하는 이유 중에 한가지
<부산지점><br>
백팩: 10 캐리어: 34 커피: 101<br>
<서울지점><br>
캐리어: 3 백팩: 55 커피: 23<br>
`2지점의 현재 수량은?`<br>
&total = 부산지점 + 서울지점<br>
&output: 백팩:65, 캐리어:37, 커피:124<br>
이와같이 스마트하게 index를 대조하여 연산이 가능하다.

In [20]:
obj = pd.Series([3, 5, -1, 2]) # index는 필수로 지정되며, 직접 지정하지 않을 경우에 0, 1, ...와 같이 지정됨
obj

0    3
1    5
2   -1
3    2
dtype: int64

In [21]:
obj.index

RangeIndex(start=0, stop=4, step=1)

In [22]:
obj = pd.Series([3, 5, -1, 2], index=['순이', '영이', '철수', '군자']) # 필요하다면 인덱스를 직접 지정
obj

순이    3
영이    5
철수   -1
군자    2
dtype: int64

In [23]:
obj.index

Index(['순이', '영이', '철수', '군자'], dtype='object')

In [24]:
obj = pd.Series({'blue': '파란색', 'red':'빨간색', 'yellow':'노란색'}) # dict 타입은 key가 index가 된다

In [25]:
obj

blue      파란색
red       빨간색
yellow    노란색
dtype: object

### key를 활용한 value접근 방법

In [26]:
obj.red

'빨간색'

In [27]:
obj['blue']

'파란색'

In [28]:
type(chart_df['곡'])

pandas.core.series.Series

In [29]:
chart_df['곡']

0                                 멍청이(twit)
1                                      달라달라
2                                       오롯이
3                           이 노래가 클럽에서 나온다면
4                이별을 걷다 (A Walk To Goodbye)
5                                    벌써 12시
6                                   7 rings
7                   신청곡 (Feat. SUGA of BTS)
8                      오랜만이야 (Feat. Zion.T)
9                         띵 (Prod. By 기리보이)
10                                     넘쳐흘러
11                           너도 그냥 날 놓아주면 돼
12                                     180도
13                                      신용재
14                                      초록빛
15                                     SOLO
16                             해야 (Sunrise)
17    모든 날, 모든 순간 (Every day, Every Moment)
18                               We all lie
19                                 MILLIONS
20                                    너를 만나
21                               YES or YES
22                            th

## 크롤링한 데이터 저장하기
위에서 크롤링한 데이터를 바로 저장해도 되지만 곡, 아티스트, 앨범만 선택하여 csv, xlsx 파일로 저장하자

In [30]:
chart_df.columns # 저장할 columns을 확인하자

Index(['Unnamed: 0', '순위', 'Unnamed: 2', 'Unnamed: 3', '곡', '아티스트', '앨범', '듣기',
       '재생목록', '내앨범', '다운', '영상', '기타', '좋아요'],
      dtype='object')

In [31]:
result_chart = chart_df[['곡', '아티스트', '앨범']] # 곡, 아티스트, 앨범을 DataFrame 으로 result_chart에 저장

### xlsx파일, csv파일로 저장하기

In [32]:
result_chart.to_excel('bugs_top_100.xlsx') # pypi 에서 제공하는 openpyxl을 설치하고 엑셀 파일로 저장

In [33]:
result_chart.to_csv('bugs_top_100.csv') # csv 파일로 저장

### 저장한 파일을 다시 불러오기

In [34]:
pd.read_csv('bugs_top_100.csv').head() # 저장한 csv 파일을 불러오기

,Unnamed: 0,곡,아티스트,앨범
0,0,멍청이(twit),화사(Hwa Sa),멍청이(twit)
1,1,달라달라,ITZY (있지),IT'z Different
2,2,오롯이,헤이즈(HEIZE),오롯이
3,3,이 노래가 클럽에서 나온다면,우디(Woody),이 노래가 클럽에서 나온다면
4,4,이별을 걷다 (A Walk To Goodbye),황치열,The Four Seasons


In [35]:
pd.read_excel('bugs_top_100.xlsx').head(10) # xlrd를 설치, 엑셀파일 불러오기

,Unnamed: 0,곡,아티스트,앨범
0,0,멍청이(twit),화사(Hwa Sa),멍청이(twit)
1,1,달라달라,ITZY (있지),IT'z Different
2,2,오롯이,헤이즈(HEIZE),오롯이
3,3,이 노래가 클럽에서 나온다면,우디(Woody),이 노래가 클럽에서 나온다면
4,4,이별을 걷다 (A Walk To Goodbye),황치열,The Four Seasons
5,5,벌써 12시,청하,벌써 12시
6,6,7 rings,Ariana Grande(아리아나 그란데),7 rings
7,7,신청곡 (Feat. SUGA of BTS),이소라,신청곡 (Feat. SUGA of BTS)
8,8,오랜만이야 (Feat. Zion.T),로꼬,HELLO
9,9,띵 (Prod. By 기리보이),Jvcki Wai Jvcki Wai,Dingo X Indigo Music


In [36]:
%ls

10minutes_to_pandas.ipynb   bugs_top_100.csv
Pipfile                     bugs_top_100.xlsx
Pipfile.lock                kaggle_titanic_train.csv
bugs.ipynb                  kaggle_titanic_train.ipynb
